In [1]:
import os
import numpy as np
import nibabel as nib
import pandas as pd
import scipy.ndimage as ndi
import seaborn as sns
import ipyvolume as ipv
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
dir_of_interest ="C:\\Users\\Shreya\\Downloads\\Car Rental\\BrainClassification"
os.chdir(dir_of_interest)

In [3]:
dataframe_original = pd.read_csv('training_data_classification_labels.csv')

In [6]:
df = dataframe_original.copy()

In [4]:
dir_of_interest ="G:\BrainClassification"
os.chdir(dir_of_interest)

In [10]:
df.head()

,CPM_RadPath_2019_ID,class,age_in_days
0,CPM19_CBICA_AAB_1,G,22393
1,CPM19_CBICA_AAG_1,G,19355
2,CPM19_CBICA_ABE_1,G,24859
3,CPM19_CBICA_ABM_1,G,25892
4,CPM19_CBICA_ABO_1,G,20896


In [15]:
y=[]
for f in os.listdir(dir_of_interest):
    if os.path.abspath(f):
        i=df.loc[df['CPM_RadPath_2019_ID']==f].index[0]
        y.append(df.iloc[i]['class'])

In [16]:
def find_max_tumor(image):
    #Takes in the numpy image and returns the layer index which contains the maximum size of tumor
    mask_sum=0
    layer_index=0
    for i in range(155): #as there are 155 height layers
        image_file=image[:,:,i]
        filt= ndi.gaussian_filter(image_file,sigma=1)
        mask = filt > 450
        labels, nlabels = ndi.label(mask)
        
        if i==154 and mask_sum==0:
            m=0
            l=0
            for j in range(155):
                image_file=image[:,:,j]
                filt= ndi.gaussian_filter(image_file,sigma=1)
                mask1 = filt > 250
                if mask1.sum() > m:
                    m=mask1.sum()
                    l=j
            mask_sum=m
            layer_index=l
        
        if mask.sum() > mask_sum:
            mask_sum=mask.sum()
            layer_index=i
    
    return layer_index

In [17]:
X=[]

for f in os.listdir(dir_of_interest):
    if os.path.abspath(f):
        flair_image="./"+f+"/"+f+"_flair.nii.gz"
        flair_image = str(flair_image)
        flair_image = nib.load(flair_image)
        #flair_image = img.get_fdata()
        flair_image = np.array(flair_image.dataobj)
        
        t1_image="./"+f+"/"+f+'_t1.nii.gz'
        t1_image = str(t1_image)
        t1_image = nib.load(t1_image)
        #t1_image = img.get_fdata()
        t1_image = np.array(t1_image.dataobj)
        
        t1ce_image="./"+f+"/"+f+'_t1ce.nii.gz'
        t1ce_image = str(t1ce_image)
        t1ce_image = nib.load(t1ce_image)
        #t1ce_image = img.get_fdata()
        t1ce_image = np.array(t1ce_image.dataobj)
        
        t2_image="./"+f+"/"+f+'_t2.nii.gz'
        t2_image = str(t2_image)
        t2_image = nib.load(t2_image)
        #t2_image = img.get_fdata()
        t2_image = np.array(t2_image.dataobj)
        
        layer_index=find_max_tumor(flair_image)
        
        flair_image=flair_image[:,:,layer_index]
        t1_image=t1_image[:,:,layer_index]
        t1ce_image=t1ce_image[:,:,layer_index]
        t2_image=t2_image[:,:,layer_index]
        #Concat the images and store into X_train
        res=np.concatenate((flair_image,t1_image,t1ce_image,t2_image),axis=1)
        img = Image.fromarray(res)
        img=img.convert('RGB')
        file_name=f
        img.save("G:/SavedImage/"+file_name+".png")
        X.append(res)

In [19]:
np.save('G:/X.npy', X)

In [20]:
np.save('G:/y.npy',y)

In [21]:
from sklearn.model_selection import train_test_split

(trainX, testX, trainY, testY) = train_test_split(X, y, test_size=0.25)

In [22]:
np.save('G:/X_train.npy', trainX)

In [23]:
np.save('G:/X_test.npy', testX)

In [24]:
np.save('G:/y_test.npy', testY)

In [25]:
np.save('G:/y_train.npy', trainY)